# Lesson 12: groupby

최규빈  
2023-07-25

<a href="https://colab.research.google.com/github/guebin/IP2023WIN/blob/main/posts/Day3/07_DataWrangling/2023-02-01-Lesson14.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# imports

In [1]:
import pandas as pd

# 판다스: groupby

## flights data

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook/master/data/flights.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58492 entries, 0 to 58491
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MONTH      58492 non-null  int64  
 1   DAY        58492 non-null  int64  
 2   WEEKDAY    58492 non-null  int64  
 3   AIRLINE    58492 non-null  object 
 4   ORG_AIR    58492 non-null  object 
 5   DEST_AIR   58492 non-null  object 
 6   SCHED_DEP  58492 non-null  int64  
 7   DEP_DELAY  57659 non-null  float64
 8   AIR_TIME   57474 non-null  float64
 9   DIST       58492 non-null  int64  
 10  SCHED_ARR  58492 non-null  int64  
 11  ARR_DELAY  57474 non-null  float64
 12  DIVERTED   58492 non-null  int64  
 13  CANCELLED  58492 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 6.2+ MB

## get_groups

`-` groupby - 데이터프레임을 여러개의 서브데이터프레임으로 나누는 기술 -
단독으로 쓸 이유는 별로 없다. $\to$ 그룹을 나누고 each 그룹마다 어떠한
“변수”에 “연산”을 하기 위함.

In [20]:
df.groupby(by="AIRLINE")

-   지금 이것이 항공사별로 데이터프레임이 나누어진 상태임

`-` 진짜 sub dataframe 으로 나누어져 있는지 확인

In [21]:
grouped = df.groupby(by="AIRLINE")
grouped.groups.keys()

In [54]:
display(grouped.get_group('AS'))

In [24]:
grouped.groups.keys()

In [26]:
#collapse_output
for key in grouped.groups.keys():
    display(grouped.get_group(key))

## 범주형변수를 기준으로 groupby -\> agg

In [33]:
df.columns

In [37]:
grouped.get_group('AA').ARR_DELAY.mean(),grouped.get_group('WN').ARR_DELAY.mean()

### `#` EX1: \[AIRLINE\] $\to$ {ARR_DELAY:mean}

`-` 방법1: grouby() $\to$ .agg({colname: function})

(예시1)

In [43]:
df.groupby(by="AIRLINE").agg({'ARR_DELAY':np.mean})

(예시2)

In [57]:
df.groupby(by="AIRLINE").agg({'ARR_DELAY':'mean'})

`-` 방법2: grouby() $\to$ key로 column선택 $\to$ .agg(f) or .f()

(예시1)

In [49]:
df.groupby(by='AIRLINE')['ARR_DELAY'].agg(np.mean)

(예시2)

In [59]:
df.groupby(by='AIRLINE')['ARR_DELAY'].agg("mean")

(예시3)

In [60]:
df.groupby(by='AIRLINE')['ARR_DELAY'].mean()

### `#` EX2: \[AIRLINE,WEEKDAY\] $\to$ {CANCELLED:sum}

`-` 방법1

(예시1)

In [61]:
df.groupby(by=["AIRLINE","WEEKDAY"]).agg({"CANCELLED":np.sum})

(예시2)

In [62]:
df.groupby(by=["AIRLINE","WEEKDAY"]).agg({"CANCELLED":"sum"})

`-` 방법2

(예시1)

In [63]:
df.groupby(by=["AIRLINE","WEEKDAY"])[["CANCELLED"]].agg(np.sum)

(예시2)

In [64]:
df.groupby(by=["AIRLINE","WEEKDAY"])["CANCELLED"].agg("sum")

(예시3)

In [65]:
df.groupby(by=["AIRLINE","WEEKDAY"])["CANCELLED"].sum()

In [66]:
df.DIVERTED

### `#` EX3: \[AIRLINE,WEEKDAY\] $\to$ {CANCELLED:sum,mean}, {DIVERTED: sum,mean}

`-` 방법1

(예시1)

In [67]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({"CANCELLED":[np.sum,np.mean],"DIVERTED":[np.sum,np.mean]})

(예시2)

In [68]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({"CANCELLED":["sum","mean"],"DIVERTED":["sum","mean"]})

`-` 방법2

(예시1)

In [69]:
df.groupby(["AIRLINE","WEEKDAY"])[["CANCELLED","DIVERTED"]]\
.agg([np.sum,np.mean])

(예시2)

In [70]:
df.groupby(["AIRLINE","WEEKDAY"])[["CANCELLED","DIVERTED"]]\
.agg(["sum","mean"])

(예시3) – 사용불가능

### `#` EX4: \[AIRLINE,WEEKDAY\] $\to$ {CANCELLED:sum,mean,count}, {AIR_TIME: mean,var}

`-` 방법1

(예시1)

In [71]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({'CANCELLED':[np.sum,np.mean,len],'AIR_TIME':[np.mean,np.var]})

(예시2)

In [72]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({'CANCELLED':["sum","mean","count"],'AIR_TIME':["mean","var"]})

(사용자정의함수)

In [73]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({'CANCELLED':[np.sum,np.mean,len],
      'AIR_TIME':[np.mean,lambda x: np.std(x,ddof=1)**2]})

## 연속형변수를 기준으로 groupby -\> agg

In [74]:
df.T

In [75]:
df.DIST.describe()

In [76]:
df.assign(DIST2 = pd.cut(df.DIST,[-np.inf,391,690,1199,np.inf]))\
.groupby(["AIRLINE","DIST2"]).agg({'CANCELLED':["sum","mean","count"]})

In [77]:
pd.cut(df.DIST,[-np.inf,400,700,1200,np.inf],labels=['~400','400~700','700~1200','1200~'])

In [78]:
df.assign(DIST2 = pd.cut(df.DIST,[-np.inf,400,700,1200,np.inf],labels=['~400','400~700','700~1200','1200~']))\
.groupby(["AIRLINE","DIST2"]).agg({'CANCELLED':["sum","mean","count"]})

## HRDataset_v14 자료분석

아래의 코드를 활용하여 Kaggle의
[HRdataset](https://www.kaggle.com/datasets/rhuebner/human-resources-data-set)을
불러오라.

In [50]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/HRDataset_v14.csv')
df

In [53]:
df.groupby('RaceDesc')

데이터를 조사하고 올바르게 분석한 사람을 모두 고르라. (모두 맞칠경우만
정답으로 인정)

-   소윤: 근무인원수가 가장 많은 인종(RaceDesc)은 ’White’이며 이는
    ’Asian’인종과 ’Black or African American’의 합보다 많다.
-   다호: ’RaceDesc==White’의 성별(Sex)임금차이는 2000이상이다.
-   하니: 퇴직한사람(Termd==1)은 모두 104명이며 백인여성의 퇴직자수가
    가장 많다.
-   도한: 퇴직한사람중 아시아인의 비율은 10%가 넘지 않는다.

(풀이)

모두 참이다.

**데이터조사**

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employee_Name               311 non-null    object 
 1   EmpID                       311 non-null    int64  
 2   MarriedID                   311 non-null    int64  
 3   MaritalStatusID             311 non-null    int64  
 4   GenderID                    311 non-null    int64  
 5   EmpStatusID                 311 non-null    int64  
 6   DeptID                      311 non-null    int64  
 7   PerfScoreID                 311 non-null    int64  
 8   FromDiversityJobFairID      311 non-null    int64  
 9   Salary                      311 non-null    int64  
 10  Termd                       311 non-null    int64  
 11  PositionID                  311 non-null    int64  
 12  Position                    311 non-null    object 
 13  State                       311 non

-   EmpID가 missing이 없는 열임

***소윤: 근무인원수가 가장 많은 인종(RaceDesc)은 ’White’이며 이는
’Asian’인종과 ’Black or African American’의 합보다 많다. — 참***

In [79]:
df.groupby(by='RaceDesc').agg({'EmpID':len})

In [80]:
29+80

***다호: ’RaceDesc==White’의 성별(Sex)임금차이는 2000이상이다. — 참***

In [81]:
df.groupby(by=['RaceDesc','Sex']).agg({'Salary':np.mean})

In [82]:
68846.519231 - 65334.132530

***하니: 퇴직한사람(Termd==1)은 모두 104명이며 백인여성의 퇴직자수가
가장 많다. — 참***

In [83]:
(df.Termd==1).sum()

In [84]:
df.groupby(by=['RaceDesc','Sex']).agg({'Termd':np.sum})

***도한: 퇴직한사람중 아시아인의 비율은 10%가 넘지 않는다. — 참***

In [85]:
(df.Termd==1).sum()

In [86]:
df.groupby(by=['RaceDesc']).agg({'Termd':np.sum})